In [4]:
import kwant
import numpy as np
import matplotlib.pyplot as plt
import tinyarray
import ipywidgets as widgets
from ipywidgets import interact

# 确保在Jupyter中正确显示matplotlib图像
%matplotlib inline

# --- 矩阵定义 (和您原来的一样) ---
sigma_0 = tinyarray.array([[1, 0], [0, 1]])
sigma_x = tinyarray.array([[0, 1], [1, 0]])
sigma_y = tinyarray.array([[0, -1j], [1j, 0]])
sigma_z = tinyarray.array([[1, 0], [0, -1]])

I_4= tinyarray.array(np.kron(sigma_0, sigma_0))
gamma_1 = tinyarray.array(np.kron(sigma_0, sigma_z))
gamma_2 = tinyarray.array(np.kron(sigma_z, sigma_z))
gamma_3 = tinyarray.array(np.kron(sigma_0, sigma_y))
gamma_4 = tinyarray.array(np.kron(sigma_0, sigma_x))

# --- 晶格定义 ---
las_val = 1
structure=kwant.lattice.square(las_val, norbs=4)
model_sym= kwant.TranslationalSymmetry(structure.vec((-1,0)),structure.vec((0,-1)))

# --- 参数化的哈密顿量函数 ---
def onsite(site, m, B, las):
    return m/2*gamma_1 - (4*B)/las**2 * gamma_2

def hopping_x(site1, site2, g, B, las):
    return B/(las**2)*gamma_2 + 1j*g/(2*las)*gamma_3

def hopping_y(site1, site2, g, B, las):
    return B/(las**2)*gamma_2 - 1j*g/(2*las)*gamma_4

# --- model_MTI (不变) ---
def model_MTI():
    model_1= kwant.Builder(model_sym)
    model_1[structure(0, 0)] = onsite
    model_1[kwant.builder.HoppingKind((1,0), structure)]  = hopping_x
    model_1[kwant.builder.HoppingKind((0,1), structure)]  = hopping_y
    return model_1

# --- model_EG (不变) ---
def model_EG():
    onsite_leads= 150 *I_4
    hopping_leads= -150 *I_4
    model_2= kwant.Builder(model_sym)
    model_2[structure(0, 0)] = onsite_leads
    model_2[structure.neighbors()]  = hopping_leads
    return model_2

# --- make_syst (不变) ---
def make_syst(width=50,length=100,leads_model=model_EG()):
    def shape_center(site):
        x, y = site.pos
        return 0 < x <= length and 0 < y <= width
    syst= kwant.Builder()
    syst.fill(model_MTI(), shape_center,(1,1))
    width_lead = length//5
    lead0= kwant.Builder(kwant.TranslationalSymmetry((-1,0)))
    lead1= kwant.Builder(kwant.TranslationalSymmetry((0,1)))
    lead2= kwant.Builder(kwant.TranslationalSymmetry((0,1)))
    def shape_lead0(site):
        x, y = site.pos
        return 0 < y <= width
    def shape_lead1(site):
        x, y = site.pos
        return width_lead < x <= 2*width_lead
    def shape_lead2(site):
        x, y = site.pos
        return 3*width_lead < x <= 4*width_lead
    lead0.fill(leads_model, shape_lead0, (0, 1))
    lead1.fill(leads_model, shape_lead1, (width_lead+1, 0))
    lead2.fill(leads_model, shape_lead2, (3*width_lead+1, 0))
    for lead in [lead0,lead1,lead2, lead0.reversed(),lead2.reversed(),lead1.reversed()]:
        syst.attach_lead(lead)
    return syst.finalized()

# --- 交互式绘图部分 (已修正) ---

# 1. 定义 syst_lead_template
print("正在构建系统，请稍候...")
syst_lead_template = make_syst(width=60, leads_model=model_MTI()).leads[0]
print("系统构建完毕，正在启动交互界面...")

# 2. 定义一个将被 widgets 调用的绘图函数
def plot_interactive_bands(m, B, g):
    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    fig.suptitle('Interactive Band Structure')
    
    xlim = 0.05
    ylim = 20
    params = dict(m=m, B=B, g=g, las=las_val)
    
    momenta = np.linspace(-xlim, xlim, 100)
    
    # --- [步骤 A] 我们自己计算能量，目的是为了获取 k=0 的数据
    bands_computer = kwant.physics.Bands(syst_lead_template, params=params)
    energies_list = [bands_computer(k) for k in momenta]
    energies = np.array(energies_list)
    
    # 找到 k=0 对应的索引
    k0_index = np.argmin(np.abs(momenta))
    energies_at_k0 = energies[k0_index]
    
    # 筛选导带
    positive_energies = np.sort(energies_at_k0[energies_at_k0 > 1e-6])
    unique_energies = np.unique(np.round(positive_energies, decimals=2))
    
    # --- [步骤 B] 我们让 kwant.plotter.bands 再次计算并画图
    # [!!! 这是最终的修正 !!!]
    # 我们不传递 'energies=energies'，因为 kwant.plotter.bands 不接受它
    kwant.plotter.bands(syst_lead_template, momenta=momenta, 
                      ax=ax, params=params) 
                      
    ax.set_ylabel('Energy (meV)')
    ax.set_xlabel('k (nm^-1)')
    ax.set_ylim(-ylim, ylim)
    ax.set_xlim(-xlim, xlim)
    ax.set_title(f'm={m:.1f}, B={B:.1f}, g={g:.1f}')
    
    # --- [步骤 C] 我们使用 [步骤 A] 得到的数据来添加文本
    text_x_position = 0.005 
    
    if len(unique_energies) > 0:
        c1_energy = unique_energies[0]
        ax.text(text_x_position, c1_energy, f'C1: {c1_energy:.2f}', 
                verticalalignment='bottom', color='red', fontsize=10, weight='bold')
        ax.axhline(y=c1_energy, xmin=0.5, xmax=0.6, color='red', linestyle='--', lw=1.5)

    if len(unique_energies) > 1:
        c2_energy = unique_energies[1]
        ax.text(text_x_position, c2_energy, f'C2: {c2_energy:.2f}', 
                verticalalignment='bottom', color='blue', fontsize=10, weight='bold')
        ax.axhline(y=c2_energy, xmin=0.5, xmax=0.6, color='blue', linestyle='--', lw=1.5)
    
    plt.show()

# 3. 使用 interact 创建交互式滑块
interact(plot_interactive_bands, 
         m=widgets.FloatSlider(value=0, min=-100, max=100, step=1, description='m:'),
         B=widgets.FloatSlider(value=-300, min=-500, max=500, step=10, description='B:'),
         g=widgets.FloatSlider(value=300, min=0, max=500, step=10, description='g:'));

正在构建系统，请稍候...
系统构建完毕，正在启动交互界面...


interactive(children=(FloatSlider(value=0.0, description='m:', min=-100.0, step=1.0), FloatSlider(value=-300.0…

In [ ]:
import kwant
import numpy as np
import matplotlib.pyplot as plt
import tinyarray
import ipywidgets as widgets
from ipywidgets import interact

# 确保在Jupyter中正确显示matplotlib图像
%matplotlib inline

# --- 矩阵定义 (和您原来的一样) ---
sigma_0 = tinyarray.array([[1, 0], [0, 1]])
sigma_x = tinyarray.array([[0, 1], [1, 0]])
sigma_y = tinyarray.array([[0, -1j], [1j, 0]])
sigma_z = tinyarray.array([[1, 0], [0, -1]])

I_4= tinyarray.array(np.kron(sigma_0, sigma_0))
gamma_1 = tinyarray.array(np.kron(sigma_0, sigma_z))
gamma_2 = tinyarray.array(np.kron(sigma_z, sigma_z))
gamma_3 = tinyarray.array(np.kron(sigma_0, sigma_y))
gamma_4 = tinyarray.array(np.kron(sigma_0, sigma_x))

# --- 晶格定义 ---
las_val = 1
structure=kwant.lattice.square(las_val, norbs=4)
model_sym= kwant.TranslationalSymmetry(structure.vec((-1,0)),structure.vec((0,-1)))

# --- 参数化的哈密顿量函数 ---
def onsite(site, m, B, las):
    return m/2*gamma_1 - (4*B)/las**2 * gamma_2

def hopping_x(site1, site2, g, B, las):
    return B/(las**2)*gamma_2 + 1j*g/(2*las)*gamma_3

def hopping_y(site1, site2, g, B, las):
    return B/(las**2)*gamma_2 - 1j*g/(2*las)*gamma_4

# --- model_MTI (不变) ---
def model_MTI():
    model_1= kwant.Builder(model_sym)
    model_1[structure(0, 0)] = onsite
    model_1[kwant.builder.HoppingKind((1,0), structure)]  = hopping_x
    model_1[kwant.builder.HoppingKind((0,1), structure)]  = hopping_y
    return model_1

# --- model_EG (不变) ---
def model_EG():
    onsite_leads= 150 *I_4
    hopping_leads= -150 *I_4
    model_2= kwant.Builder(model_sym)
    model_2[structure(0, 0)] = onsite_leads
    model_2[structure.neighbors()]  = hopping_leads
    return model_2

# --- make_syst (不变) ---
def make_syst(width=50,length=100,leads_model=model_EG()):
    def shape_center(site):
        x, y = site.pos
        return 0 < x <= length and 0 < y <= width
    syst= kwant.Builder()
    syst.fill(model_MTI(), shape_center,(1,1))
    width_lead = length//5
    lead0= kwant.Builder(kwant.TranslationalSymmetry((-1,0)))
    lead1= kwant.Builder(kwant.TranslationalSymmetry((0,1)))
    lead2= kwant.Builder(kwant.TranslationalSymmetry((0,1)))
    def shape_lead0(site):
        x, y = site.pos
        return 0 < y <= width
    def shape_lead1(site):
        x, y = site.pos
        return width_lead < x <= 2*width_lead
    def shape_lead2(site):
        x, y = site.pos
        return 3*width_lead < x <= 4*width_lead
    lead0.fill(leads_model, shape_lead0, (0, 1))
    lead1.fill(leads_model, shape_lead1, (width_lead+1, 0))
    lead2.fill(leads_model, shape_lead2, (3*width_lead+1, 0))
    for lead in [lead0,lead1,lead2, lead0.reversed(),lead2.reversed(),lead1.reversed()]:
        syst.attach_lead(lead)
    return syst.finalized()

# --- 交互式绘图部分 (已修正) ---

# 1. 定义 syst_lead_template
print("正在构建系统，请稍候...")
syst_lead_template = make_syst(width=60, leads_model=model_MTI()).leads[0]
print("系统构建完毕，正在启动交互界面...")

# 2. 定义一个将被 widgets 调用的绘图函数
def plot_interactive_bands(m, B, g):
    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    fig.suptitle('Interactive Band Structure')
    
    xlim = 0.05
    ylim = 20
    params = dict(m=m, B=B, g=g, las=las_val)
    
    momenta = np.linspace(-xlim, xlim, 100)
    
    # --- [步骤 A] 我们自己计算能量，目的是为了获取 k=0 的数据
    bands_computer = kwant.physics.Bands(syst_lead_template, params=params)
    energies_list = [bands_computer(k) for k in momenta]
    energies = np.array(energies_list)
    
    # 找到 k=0 对应的索引
    k0_index = np.argmin(np.abs(momenta))
    energies_at_k0 = energies[k0_index]
    
    # 筛选导带
    positive_energies = np.sort(energies_at_k0[energies_at_k0 > 1e-6])
    unique_energies = np.unique(np.round(positive_energies, decimals=2))
    
    # --- [步骤 B] 我们让 kwant.plotter.bands 再次计算并画图
    kwant.plotter.bands(syst_lead_template, momenta=momenta, 
                      ax=ax, params=params) 
                      
    ax.set_ylabel('Energy (meV)')
    ax.set_xlabel('k (nm^-1)')
    ax.set_ylim(-ylim, ylim)
    ax.set_xlim(-xlim, xlim)
    ax.set_title(f'm={m:.1f}, B={B:.1f}, g={g:.1f}')
    
    # --- [步骤 C] 我们使用 [步骤 A] 得到的数据来添加文本
    # [!!! 标签位置已修改 !!!]
    
    if len(unique_energies) > 0:
        c1_energy = unique_energies[0]
        # 标注在右侧固定位置
        ax.text(0.95, 0.70, f'C1: {c1_energy:.2f}', 
                transform=ax.transAxes, # 使用-坐标系
                horizontalalignment='right', # 右对齐
                verticalalignment='center',
                color='red', fontsize=12, weight='bold')
        # 指向能带的短线 (不变)
        ax.axhline(y=c1_energy, xmin=0.5, xmax=0.6, color='red', linestyle='--', lw=1.5)

    if len(unique_energies) > 1:
        c2_energy = unique_energies[1]
        # 标注在右侧固定位置
        ax.text(0.95, 0.60, f'C2: {c2_energy:.2f}', 
                transform=ax.transAxes, # 使用-坐标系
                horizontalalignment='right', # 右对齐
                verticalalignment='center',
                color='blue', fontsize=12, weight='bold')
        # 指向能带的短线 (不变)
        ax.axhline(y=c2_energy, xmin=0.5, xmax=0.6, color='blue', linestyle='--', lw=1.5)
    
    plt.show()

# 3. 使用 interact 创建交互式滑块
interact(plot_interactive_bands, 
         m=widgets.FloatSlider(value=0, min=-100, max=100, step=1, description='m:'),
         B=widgets.FloatSlider(value=-300, min=-500, max=500, step=10, description='B:'),
         g=widgets.FloatSlider(value=300, min=0, max=500, step=10, description='g:'));

正在构建系统，请稍候...
系统构建完毕，正在启动交互界面...


interactive(children=(FloatSlider(value=0.0, description='m:', min=-100.0, step=1.0), FloatSlider(value=-300.0…